In [1]:
import requests
from bs4 import BeautifulSoup
from twilio.rest import Client

ACCOUNT_SID='AC48502b4b2305275af8d653d8f3bc2ad5'
AUTH_TOKEN='2bcd456382adaf6085a3b4c9f8453db4'


def get_score():
    url = 'https://www.cricbuzz.com/cricket-match/live-scores'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    match_blocks = soup.find_all('div', class_='cb-mtch-lst cb-col cb-col-100 cb-tms-itm')

    for match in match_blocks:  
        try:
            match_link = 'https://www.cricbuzz.com' + match.find('h3', class_='cb-lv-scr-mtch-hdr').find('a')['href']
            match_id=match_link.split('/')[4]
            cricbuzz_api=f'https://www.cricbuzz.com/api/cricket-match/commentary/{match_id}'
            res=requests.get(cricbuzz_api)
            commentary=res.json()['commentaryList']
            for comm in commentary:
                try:
                    score=comm['overSeparator']['score']
                    wickets=comm['overSeparator']['wickets']
                    summary=comm['overSeparator']['o_summary']
                    over=comm['overNumber']
                    batteam=comm['batTeamName']
                    break
                except:
                    score=wickets=summary=over=batteam=None
            result=f'{batteam} {score}/{wickets} ({over})      This Over:{summary}'
            return result,over

        except Exception as e:
            print(f"Error parsing match: {e}")
    return

def send_message(message,over):
    if str(over)[-1]=='6':
        client=Client(ACCOUNT_SID,AUTH_TOKEN)
        client.messages.create(from_='whatsapp:+14155238886',to='whatsapp:+94772153646',body=message)

def main():
    result=get_score()
    if result:
        message,over=result
        send_message(message,over)

main()
